In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%gui qt

In [4]:
import numpy as np

In [5]:
from spiketag.base import *
from spiketag.view import *

In [6]:
from hdbscan import HDBSCAN

In [7]:
import phy
from phy.gui import GUI

In [8]:
filename  = '/tmp_data/pcie.bin'
nCh       = 32
fs        = 25000
numbyte   = 4
time_span = 1 # 1 seconds
global time_slice
time_slice = 0
span = time_span * fs
highlight = True

In [9]:
mua = MUA(filename=filename, nCh=nCh, fs=fs, numbytes=numbyte)
spk = mua.tospk(ch_span=2)
fet = spk.tofet()

#############  load data  ###################
/tmp_data/pcie.bin loaded, it contains: 
30150587 * 32 points (3859275136 bytes) 
32 channels with sampling rate of 25000.0000 
1206.023 secs (20.100 mins) of data
#############################################


# test spike_view

In [11]:
spk_view = spike_view()

In [12]:
spk_view.set_data(spk[26])

In [14]:
spk_view.show()

In [15]:
spk_view.highlight(np.arange(25),0)

In [19]:
spk_view.selected_spk

array([    0,     4,     9, ..., 13054, 13056, 13057])

In [20]:
spk_view_2 = spike_view()

In [21]:
spk_view_2.set_data(spk[30])

In [22]:
spk_view_2.show()

# start GUI

In [23]:
gui = GUI(position=(0, 0), size=(600, 400), name='GUI')

In [24]:
props = param_widget()
gui.add_view(props,position='left', name='params')

In [25]:
scatter_view = scatter_3d_view()
scatter_view.unfreeze()
gui.add_view(scatter_view)
spk_view = spike_view()
gui.add_view(spk_view)

In [27]:
wave_view = wave_view(mua.data[time_slice*span:(time_slice+1)*span,:], fs=25000)
wave_view.gap_value = 0.9
wave_view.attach(gui)

In [28]:
from phy.gui import Actions
actions = Actions(gui)

@actions.add(shortcut=',')
def page_up():
#     print 'page_up'
    global time_slice
    time_slice -= 1
    if time_slice >= 0:
        wave_view.set_data(mua.data[time_slice*span:(time_slice+1)*span,:], time_slice)
    else:
        time_slice = 0

@actions.add(shortcut='.')
def page_down():
#     print 'page_down'
    global time_slice
    time_slice += 1
    wave_view.set_data(mua.data[time_slice*span:(time_slice+1)*span,:], time_slice)

In [29]:
clu = fet.toclu('hdbscan', njobs=24)

clustering finished, used 61.6415410042 seconds


In [30]:
cluster = {}
for key, value in clu.items():
    cluster[key] = CLU(value)

In [31]:
def get_time_slice(tpos):
    global span
    ts = int(np.floor(float(tpos)/span))
    return ts

In [32]:
def init_waveform(ch):
    wave_view.waves1.highlight_reset()
    highlight_ch = mua.get_near_ch(ch, ch_span=2)
    chlist = np.delete(np.arange(32), highlight_ch)
    wave_view.waves1.highlight_ch(ch=chlist,
                                  highlight_color=(0.25,0.25,0.25,0.5))
    return highlight_ch

In [33]:
def update_params():
    
    # ---- update chosen ch and get cluster ----
    global ch
    ch = props.ch.value()
    clu = cluster[ch]
    
    # ---- get spike timelist for chosen ch ----
    timelist = []
    timelist = mua.pivotal_pos[mua.pivotal_pos[:,1]==ch][:,0]
#     print len(timelist)
    
    # ---- set scatter view and spk_view ----
    scatter_view.set_data(fet=fet[ch], clu=clu.membership)
    spk_view.set_data(spk[ch], clu)
    
    # ---- set waveform view ----
    highlight_ch = init_waveform(ch)
    # --------
    
    
    
    @clu.connect
    def on_cluster(*args, **kwargs):
        scatter_view.set_data(fet=fet[ch], clu=clu.membership)


    @spk_view.connect_
    def on_select(*args, **kwargs):
    #         print kwargs['global_idx']
        global_idx = kwargs['global_idx']
        scatter_view.highlight(global_idx)
    #         if len(global_idx) == 1:
    #             print timelist[global_idx]

    @spk_view.connect_
    def on_click(*args, **kwargs):
        global ch
        timelist = mua.pivotal_pos[mua.pivotal_pos[:,1]==ch][:,0]
        global_idx = kwargs['global_idx']
        tpos = timelist[global_idx]
        ts = get_time_slice(tpos)
        tpos_offset = tpos-ts*span

        wave_view.set_data(mua.data[ts*span:(ts+1)*span,:], ts)
        highlight_ch = init_waveform(ch)
        wave_view.waves1.highlight(spacial_code=highlight_ch, 
                                  temporal_code=((tpos_offset-10,tpos_offset+10),), 
                                  highlight_color=np.hstack((spk_view.palette[spk_view.selected_clu], 1)))

    # wave_view update
#     highlight_list = mua.spk_peak_pos
#     global time_slice
#     sublist = np.logical_and(highlight_list[:,0]<(time_slice+1)*span, 
#                              highlight_list[:,0]>(time_slice)*span)
#     mask_n = highlight_list[sublist][:,0] - time_slice*span
#     mask_ch = highlight_list[sublist][:,1]
#     mask = np.hstack((mask_n.reshape(-1,1), mask_ch.reshape(-1,1)))

#     wave_view.waves1.highlight_reset()
#     for clu_id in np.unique(clu):
#         color = palette[clu_id] if clu_id>=0 else np.array([1,1,1])
#         wave_view.waves1.highlight_spikes(mask[np.logical_and(clu==clu_id, mask_ch=ch)], color = np.hstack((color, 1)))


In [34]:
update_params()
props.signal_objet_changed.connect(update_params)

In [35]:
gui.show()

  File "/usr/local/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/anaconda2/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 827, in start
    self._run_callback(callback)
  File "/usr/local/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 600, in _run_c

In [24]:
spk_view.rerender(data_bound=(-1,-0.8,1,0.8))

In [41]:
spk_view.highlight([0,1,2],0)

In [43]:
spk_view.show()

In [44]:
gui.show()

In [22]:
view_mask = spk_view._spkNo2maskNo(spkNolist=spk_view._spkNolist, cluNo=spk_view.selected_clu)

In [1]:
from spiketag.utils import Timer
import numexpr as ne
from numba import int32, int64, float32, jit

In [24]:
@jit('(int64[:], float32[:,:], float32[:,:])')
def _cache_in(mask, source, target):
    N = len(mask)
    M = target.shape[1]
    for i in xrange(N):
        k = mask[i]
        for j in xrange(M):
            target[k,j] = source[i,j]

In [25]:
from numba import int32, int64, float32, jit

@jit('(int64[:], float32[:,:], float32[:,:])')
def _cache_out(_cache_mask, _cache, target):
    N = len(_cache_mask)
    M = target.shape[1]
    for i in xrange(N):
        k = _cache_mask[i]
        for j in xrange(M):
            target[k,j] = _cache[k,j]

In [26]:
def rerender():
    if len(spk_view._cache_mask_)>0:
        with Timer('render cache mask'):
            _cache_mask_ = spk_view._cache_mask_
#             spk_view.color[_cache_mask_, :3] = spk_view._cache_color[_cache_mask_, :3]
            _cache_out(spk_view._cache_mask_, spk_view._cache_color[:,:3], spk_view.color[:,:3])
#             spk_view.color[_cache_mask_,-1] = spk_view._transparency
#             spk_view.depth[_cache_mask_, 2] = spk_view._cache_depth[_cache_mask_, 2]
            _cache_out(spk_view._cache_mask_, spk_view._cache_depth, spk_view.depth)
            spk_view._cache_mask_ = np.array([])

    if len(view_mask)>0:
        with Timer('render view mask'):
            _highlight_color = np.tile(spk_view._highlight_color, (len(view_mask),1))
            _cache_in(view_mask, _highlight_color, spk_view.color)
#             spk_view.color[view_mask, :] = spk_view._highlight_color
            _depth = np.tile(-1, (len(view_mask),1)).astype('float32')
            _cache_in(view_mask, _depth, spk_view.depth[:,2].reshape(-1,1))
#             spk_view.depth[view_mask, 2] = -1
            spk_view._cache_mask_ = np.hstack((spk_view._cache_mask_, view_mask)).astype('int64')
#             spk_view.color[view_mask,-1] = 1
            spk_view.visuals[0].program['a_color']    = spk_view.color
            spk_view.visuals[0].program['a_position'] = spk_view.depth  # pos include depth
            spk_view.update()

f:\spiketag\spiketag\view\MyWaveVisual.py:192: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  self.color[start:end,:] = np.asarray(highlight_color)


get_closest_spikes, elapsed time: 0.839556451449 ms
highlight, elapsed time: 0.190901589203 ms
on_select, elapsed time: 0.24755625439 ms
get_closest_spikes, elapsed time: 0.157237222936 ms
highlight, elapsed time: 0.0394119409961 ms
on_select, elapsed time: 0.14245774506 ms
get_closest_spikes, elapsed time: 0.125625561928 ms
highlight, elapsed time: 0.0472122209842 ms
on_select, elapsed time: 0.148205319793 ms
get_closest_spikes, elapsed time: 0.151900189259 ms
highlight, elapsed time: 0.0488543851951 ms
on_select, elapsed time: 0.168321831339 ms
get_closest_spikes, elapsed time: 0.137120711386 ms
highlight, elapsed time: 0.0435173515179 ms
on_select, elapsed time: 0.15805830504 ms
get_closest_spikes, elapsed time: 0.134657465075 ms
highlight, elapsed time: 0.037359235737 ms
on_select, elapsed time: 0.140405039801 ms
get_closest_spikes, elapsed time: 0.122751774562 ms
highlight, elapsed time: 0.0357170715262 ms
on_select, elapsed time: 0.140405039804 ms
get_closest_spikes, elapsed time

In [29]:
highlight_color = np.tile(spk_view._highlight_color, (len(view_mask),1))

array([[ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.],
       ..., 
       [ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  1.]], dtype=float32)

In [26]:
%timeit cache_out(spk_view._cache_mask_, spk_view._cache_color[:,:3], spk_view.color[:,:3])

1000 loops, best of 3: 1.68 ms per loop


In [27]:
%timeit cache_out(spk_view._cache_mask_, spk_view._cache_depth, spk_view.depth)

1000 loops, best of 3: 1.64 ms per loop


In [ ]:
cache_out(spk_view._cache_mask_, spk_view._cache_color[:,:-1], spk_view.color[:,:3])

In [59]:
spk_view._cache_color.shape

(1726000L, 4L)

In [59]:
spk_view.depth[:,2]

dtype('float32')

In [57]:
_cache_in(view_mask, _depth, spk_view.depth[:,2].reshape(-1,1))

TypeError: No matching definition for argument type(s) array(int64, 1d, C), array(int32, 2d, C), array(float32, 2d, A)

In [50]:
%timeit spk_view.color[_cache_mask_, :3] = spk_view._cache_color[_cache_mask_, :3]

100 loops, best of 3: 17.6 ms per loop


In [52]:
_cache_mask_.shape, view_mask.shape

((432500L,), (432500L,))

In [44]:
spk_view.color.shape, spk_view.depth.shape

((1726000L, 4L), (1726000L, 3L))

In [ ]:
%timeit rerender()

In [35]:
rerender()

render cache mask, elapsed time: 6.77639060518 ms
render view mask, elapsed time: 10.1255845078 ms


In [30]:
clu = cluster[29]

In [31]:
clu._registered_func_name('cluster')

['spiketag.view.spike_view.on_cluster', '__main__.on_cluster']

get_subset, elapsed time: 0.310369035333 ms
reassign membership, elapsed time: 0.00533703359906 ms
reconstruct, elapsed time: 0.706951691654 ms
renew, elapsed time: 588.389899048 ms
emit move signal, elapsed time: 599.513098312 ms
move, elapsed time: 600.74143714 ms
highlight, elapsed time: 3.03759324413 ms
get_subset, elapsed time: 0.276294128071 ms
reassign membership, elapsed time: 0.00451595155937 ms
reconstruct, elapsed time: 0.647012697982 ms
renew, elapsed time: 181.893086899 ms
emit move signal, elapsed time: 185.947590329 ms
move, elapsed time: 187.077809846 ms
highlight, elapsed time: 3.10369035356 ms
get_subset, elapsed time: 0.107972296746 ms
reassign membership, elapsed time: 0.00328432838614 ms
reconstruct, elapsed time: 0.649065403309 ms
renew, elapsed time: 181.634035495 ms
emit move signal, elapsed time: 185.723024374 ms
move, elapsed time: 186.684511518 ms
highlight, elapsed time: 0.157237222993 ms
get_subset, elapsed time: 0.0940139009344 ms
reassign membership, elap

In [27]:
%timeit spk_view.renew(spk_view.clu)

1 loop, best of 3: 590 ms per loop


In [23]:
for cls, data_list in spk_view._items.items():
    print cls.allow_list

('x', 'y')


In [34]:
for cls, data_list in spk_view._items.items():
    print cls
    print data_list[0].keys()
    data_list = data_list
    #     data = _accumulate(data_list, cls.allow_list)
#     box_index = data.pop('box_index')
#     visual = cls()
#     self.add_visual(visual)

<class 'phy.plot.visuals.PlotVisual'>
['color', 'box_index', 'depth', 'data_bounds', 'y', 'x']


In [31]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [32]:
from spiketag.view._core import _accumulate

In [27]:
y = data_list[0]['y'].pop(2)

array([ 0.17689535,  0.17322454,  0.17434254,  0.17714804,  0.17874058,
        0.17977639,  0.1797881 ,  0.18188739,  0.18649863,  0.19461696,
        0.20460857,  0.21118361,  0.21471013,  0.21385491,  0.21084124,
        0.20942864,  0.21092717,  0.21575017,  0.22309922,  0.23153883], dtype=float32)

In [34]:
x = data_list[0]['x'].pop(2)

In [33]:
data_list[1]['y'].append(y)

In [35]:
data_list[1]['x'].append(x)

In [40]:
data.keys()

['y', 'depth', 'data_bounds', 'color', 'x', 'box_index']

In [64]:
spk_view.clu.nclu

5

In [59]:
def get_new_data_list(clu):
    spk_view.clu = clu
    nclu = clu.nclu
    base_x = spk_view._xsig
    data_bound = (-1,-1,1,1)
    new_data_list=[]
    for chNo in range(spk_view.n_ch):
        for cluNo in clu.index_id:
    #         i = cluNo + chNo*nclu
            y           = list(spk_view.spk[clu[cluNo],:,chNo].squeeze())
            x           = [base_x for i in y]
            n           = len(x)
            color       = np.hstack((np.array([spk_view.palette[cluNo] for i in y]), spk_view.transparency*np.ones((n,1)))).astype('float32')
            depth       = np.zeros((n,1))
            data_bounds = data_bound*np.ones((n,1))
            box_index   = [chNo, cluNo] * np.ones((n*spk_view.n_samples,1))
            new_data_list.append({'y': y,
                                  'x': x,
                                  'color': color,
                                  'depth': depth,
                                  'data_bounds': data_bounds,
                                  'box_index': box_index})
    data = _accumulate(new_data_list, cls.allow_list)
    return data

In [214]:
new_data_list = get_new_data_list()

In [300]:
data_list[1]['data_bounds']

array([[-2., -1.,  2.,  1.],
       [-2., -1.,  2.,  1.],
       [-2., -1.,  2.,  1.],
       [-2., -1.,  2.,  1.],
       [-2., -1.,  2.,  1.],
       [-2., -1.,  2.,  1.]])

In [216]:
data_list[1]['y']

[array([ 0.44885188,  0.47664273,  0.50341558,  0.52376956,  0.5354616 ,
         0.5395686 ,  0.54003215,  0.54179615,  0.54502165,  0.54617512,
         0.53302944,  0.48971528,  0.40475905,  0.27475896,  0.11070183,
        -0.06439477, -0.21941063, -0.32489482, -0.36282966, -0.33210662], dtype=float32),
 array([ 0.41780972,  0.44758952,  0.47871834,  0.50587004,  0.52410734,
         0.53385615,  0.53869677,  0.54561603,  0.55807948,  0.57404965,
         0.58333993,  0.5678817 ,  0.50818175,  0.39219728,  0.22338104,
         0.02091879, -0.18181095, -0.34657553, -0.44198599, -0.45377442], dtype=float32),
 array([ 0.40008694,  0.41793847,  0.44110286,  0.46556252,  0.48605844,
         0.50092757,  0.51191097,  0.52087885,  0.53290087,  0.54751205,
         0.5570637 ,  0.54770899,  0.50239396,  0.40871266,  0.26670167,
         0.08904949, -0.09809366, -0.26120874, -0.37171611, -0.41143104], dtype=float32),
 array([ 0.44105381,  0.46444166,  0.48286858,  0.4938646 ,  0.49929613,


In [217]:
data_list[1]['color']

array([[ 0.18039216,  0.80000001,  0.44313726,  0.2       ],
       [ 0.18039216,  0.80000001,  0.44313726,  0.2       ],
       [ 0.18039216,  0.80000001,  0.44313726,  0.2       ],
       [ 0.18039216,  0.80000001,  0.44313726,  0.2       ],
       [ 0.18039216,  0.80000001,  0.44313726,  0.2       ],
       [ 0.18039216,  0.80000001,  0.44313726,  0.2       ]], dtype=float32)

In [218]:
new_data_list[1]['x']

[array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
        -0.23684211, -0.18421053, -0.13157895, -0.07894737, -0.02631579,
         0.02631579,  0.07894737,  0.13157895,  0.18421053,  0.23684211,
         0.28947368,  0.34210526,  0.39473684,  0.44736842,  0.5       ]),
 array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
        -0.23684211, -0.18421053, -0.13157895, -0.07894737, -0.02631579,
         0.02631579,  0.07894737,  0.13157895,  0.18421053,  0.23684211,
         0.28947368,  0.34210526,  0.39473684,  0.44736842,  0.5       ]),
 array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
        -0.23684211, -0.18421053, -0.13157895, -0.07894737, -0.02631579,
         0.02631579,  0.07894737,  0.13157895,  0.18421053,  0.23684211,
         0.28947368,  0.34210526,  0.39473684,  0.44736842,  0.5       ]),
 array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
        -0.23684211, -0.18421053, -0.13157895

In [257]:
gui.show()

In [23]:
for cls, data_list in spk_view._items.items():
    print cls
    print data_list[0].keys()
    data_list = data_list

<class 'phy.plot.visuals.PlotVisual'>
['color', 'box_index', 'depth', 'data_bounds', 'y', 'x']
get_subset, elapsed time: 1.06289078376 ms
reassign membership, elapsed time: 0.0106740673544 ms
reconstruct, elapsed time: 0.633464843318 ms
emit move signal, elapsed time: 1036.48847817 ms
move, elapsed time: 1038.61549136 ms
highlight, elapsed time: 15.986879108 ms


In [305]:
data = _accumulate(data_list, cls.allow_list)

In [326]:
clu.index_id

AttributeError: 'numpy.ndarray' object has no attribute 'index_id'

In [24]:
clu = spk_view.clu.membership

In [25]:
cluster[30].index_id

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [26]:
clu = cluster[30].membership.copy()

In [27]:
clu = CLU(clu)

In [55]:
clu.index_id

array([0, 1, 2, 3, 4], dtype=int64)

get_subset, elapsed time: 0.844482944075 ms
reassign membership, elapsed time: 0.0102635263204 ms
reconstruct, elapsed time: 0.557925289741 ms
emit move signal, elapsed time: 0.00451595155937 ms
move, elapsed time: 1.73001999335 ms
highlight, elapsed time: 53.8564173725 ms


In [62]:
data = get_new_data_list(clu)

In [52]:
def renew(clu):
    
    spk_view.clu = clu
    nclu = clu.nclu
    base_x = spk_view._xsig
    data_bound = (-1,-1,1,1)
    new_data_list=[]
    for chNo in range(spk_view.n_ch):
        for cluNo in clu.index_id:
    #         i = cluNo + chNo*nclu
            y           = list(spk_view.spk[clu[cluNo],:,chNo].squeeze())
            x           = [base_x for i in y]
            n           = len(x)
            color       = np.hstack((np.array([spk_view.palette[cluNo] for i in y]), spk_view.transparency*np.ones((n,1)))).astype('float32')
            depth       = np.zeros((n,1))
            data_bounds = data_bound*np.ones((n,1))
            box_index   = [chNo, cluNo] * np.ones((n*spk_view.n_samples,1))
            new_data_list.append({'y': y,
                                  'x': x,
                                  'color': color,
                                  'depth': depth,
                                  'data_bounds': data_bounds,
                                  'box_index': box_index})
    data = _accumulate(new_data_list, cls.allow_list)
    
    spk_view.grid.shape = (spk_view.n_ch, clu.nclu)
    box_index = data.pop('box_index')

    
    spk_view.y = np.asarray(data['y']).ravel()
    spk_view.x = np.tile(spk_view._xsig, len(spk_view.y)/spk_view.n_samples)
    spk_view.depth = np.c_[spk_view.x, spk_view.y, np.zeros(*spk_view.x.shape)].astype(np.float32)
    spk_view.color = np.repeat(data['color'], spk_view.n_samples, axis=0)
    spk_view._cache_depth = spk_view.depth.copy()
    spk_view._cache_color = spk_view.color.copy()
    spk_view._cache_mask_ = np.array([])

    
    spk_view.visuals[0].set_data(**data)
    spk_view.visuals[0].program['a_box_index'] = box_index.astype(np.float32)

In [53]:
renew(data)

In [63]:
box_index = data.pop('box_index');

In [54]:
spk_view.update()

In [24]:
spk_view._spkNo2maskNo(spk_view._spkNolist, spk_view.selected_clu)

100 loops, best of 3: 13.2 ms per loop
move, elapsed time: 1214.17721415 ms
highlight, elapsed time: 27.5945168137 ms
move, elapsed time: 1147.02049831 ms
highlight, elapsed time: 27.2266720311 ms
move, elapsed time: 1113.92801573 ms
highlight, elapsed time: 27.5152823907 ms
move, elapsed time: 1113.77734717 ms
highlight, elapsed time: 28.1200093604 ms
move, elapsed time: 1115.59358078 ms
highlight, elapsed time: 27.1137732418 ms


In [42]:
# ---- recluster when fall-off-size change ----
# fall_off_size = props.clu_param.value()
min_cluster_size = 15
leaf_size = 15
hdbcluster = HDBSCAN(min_cluster_size=min_cluster_size, 
                     leaf_size=leaf_size,
                     gen_min_span_tree=True, 
                     algorithm='boruvka_kdtree')
cluster[ch].membership = hdbcluster.fit_predict(fet[ch]) + 1
cluster[ch].__construct__()
scatter_view.set_data(fet=fet[ch], clu=cluster[ch].membership)
spk_view.clear()
spk_view.set_data(spk[ch], cluster[ch])

# OPTICS

In [21]:
from pyclustering.cluster.optics import optics

In [180]:
optics_instance = optics(fet[30], 0.01, 5)

In [181]:
optics_instance.process()

In [130]:
_clu = optics_instance.get_clusters()

In [131]:
ordering = optics_instance.get_cluster_ordering()

In [132]:
indexes = [i for i in range(0, len(ordering))]

In [133]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [134]:
plt.plot(indexes, ordering)

In [135]:
from pyclustering.utils import  draw_clusters

In [136]:
draw_clusters(fet[26][:,:3],_clu,[],'.')

In [137]:
optic_clu = np.zeros_like(clu[26])
for i,_ in enumerate(_clu):
    optic_clu[_clu[i]] = i+1
optic_clu

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [138]:
optics_instance.get_noise()

[125, 344, 601, 1085, 1224, 1479, 1719, 2108, 2375]

In [118]:
view = spike_view()
view.set_data(spk[26][_clu[0]])
view.show()

In [139]:
scatter_view.set_data(fet=fet[ch], clu=optic_clu)
spk_view.clear()
spk_view.set_data(spk[ch], CLU(optic_clu))

move, elapsed time: 151.024915737 ms
highlight, elapsed time: 15.9064130617 ms
move, elapsed time: 149.384393693 ms
highlight, elapsed time: 0.559977994726 ms
move, elapsed time: 148.808404596 ms
highlight, elapsed time: 0.559156912914 ms
move, elapsed time: 149.686551907 ms
highlight, elapsed time: 0.529597957211 ms
move, elapsed time: 149.482923545 ms
highlight, elapsed time: 0.472122209885 ms
move, elapsed time: 151.325021246 ms
highlight, elapsed time: 18.3897758857 ms


# pydpc

In [140]:
from pydpc import Cluster

In [175]:
dpc = Cluster(fet[30]*15)

In [176]:
dpc.assign(500, 0.5)

In [177]:
dpc_clu = dpc.membership

In [178]:
scatter_view.set_data(fet=fet[ch], clu=dpc_clu)
spk_view.clear()
spk_view.set_data(spk[ch], CLU(dpc_clu))

# Other Temp Test

In [91]:
ch=26

In [92]:
clu[ch] = cluster[ch].membership

In [93]:
dview= scatter_3d_view()
dview.set_data(fet[ch], clu[ch])
dview.show()

In [94]:
from metric_learn import LMNN

In [95]:
lmnn = LMNN(k=3, max_iter=500, learn_rate=1e-7, verbose=True)
lmnn.fit(fet[ch], clu[ch])

1 530240.492273 -inf 14570 1e-07
2 288200.717956 -242039.774316 12830 1.01e-07
3 204672.995344 -83527.7226125 12474 1.0201e-07
4 157012.511177 -47660.4841669 13113 1.030301e-07
5 127017.861915 -29994.6492618 13770 1.04060401e-07
6 115828.203535 -11189.6583798 13860 1.0510100501e-07
7 110292.390146 -5535.81338979 13850 1.0615201506e-07
8 105557.439469 -4734.9506763 13896 1.07213535211e-07
9 101101.215681 -4456.22378855 14028 1.08285670563e-07
10 96850.6865975 -4250.52908337 14160 1.09368527268e-07
11 92810.5302294 -4040.15636809 14280 1.10462212541e-07
12 88986.2844161 -3824.24581332 14394 1.11566834667e-07
13 85356.6617612 -3629.62265486 14508 1.12682503013e-07
14 81916.6097333 -3440.05202796 14626 1.13809328043e-07
15 78663.3701911 -3253.2395422 14768 1.14947421324e-07
16 75583.6711425 -3079.69904856 14959 1.16096895537e-07
17 72679.5709076 -2904.10023495 15072 1.17257864492e-07
18 69939.9263229 -2739.64458469 15238 1.18430443137e-07
19 67363.3989904 -2576.52733243 15400 1.19614747569

In [96]:
X_lmnn = lmnn.transform(fet[ch])
y = clu[ch]
msdview= scatter_3d_view()
msdview.set_data(X_lmnn[:,0:], y)
msdview.show()

In [98]:
fet.fet[26] = X_lmnn

In [78]:
fet = spk.tofet('weighted-pca')

# LDA

In [84]:
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

In [85]:
print("Computing Linear Discriminant Analysis projection")
ch = 26
X2 = fet[ch].copy()
y  = cluster[ch].membership
lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=3)
lda.fit(X2, y)

Computing Linear Discriminant Analysis projection


LinearDiscriminantAnalysis(n_components=3, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [86]:
lda.scalings_.shape

(6L, 3L)

In [87]:
X_lda = np.dot(X2,lda.scalings_)
X_lda.shape

(2566L, 3L)

In [88]:
ldaview= scatter_3d_view()
ldaview.set_data(X_lda, y)
ldaview.show()

In [89]:
lda.scalings_.shape

(6L, 3L)

In [90]:
fet.fet[26] = X_lda